<a href="https://colab.research.google.com/github/jyothsnakopera/Predictive-Restaurant-Recommender/blob/main/Predictive_Restaurant_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

# Define paths
train_base = "/content/drive/MyDrive/recomendation system/Assignment/Train"
test_base = "/content/drive/MyDrive/recomendation system/Assignment/Test"  # define test folder path
sample_submission_path = "/content/drive/MyDrive/recomendation system/Assignment/SampleSubmission.csv"  # define sample submission path

# Load train CSVs
orders = pd.read_csv(f"{train_base}/orders.csv", low_memory=False)
train_customers = pd.read_csv(f"{train_base}/train_customers.csv")
train_locations = pd.read_csv(f"{train_base}/train_locations.csv")
vendors = pd.read_csv(f"{train_base}/vendors.csv")

# Load test CSVs
test_customers = pd.read_csv(f"{test_base}/test_customers.csv")
test_locations = pd.read_csv(f"{test_base}/test_locations.csv")
sample_submission = pd.read_csv(sample_submission_path)


In [ ]:
# --- Initial Inspection of Each DataFrame ---

print("\n--- Orders Data ---")
orders.info()
print("\nFirst 5 rows of Orders:")
print(orders.head())

print("\n--- Train Customers Data ---")
train_customers.info()
print("\nFirst 5 rows of Train Customers:")
print(train_customers.head())

print("\n--- Train Locations Data ---")
train_locations.info()
print("\nFirst 5 rows of Train Locations:")
print(train_locations.head())

print("\n--- Vendors Data ---")
vendors.info()
print("\nFirst 5 rows of Vendors:")
print(vendors.head())

print("\n--- Test Customers Data ---")
test_customers.info()
print("\nFirst 5 rows of Test Customers:")
print(test_customers.head())

print("\n--- Test Locations Data ---")
test_locations.info()
print("\nFirst 5 rows of Test Locations:")
print(test_locations.head())

print("\n--- Sample Submission File ---")
sample_submission.info()
print("\nFirst 5 rows of Sample Submission:")
print(sample_submission.head())



--- Orders Data ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135303 entries, 0 to 135302
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   order_id                        135233 non-null  float64
 1   customer_id                     135303 non-null  object 
 2   item_count                      128378 non-null  float64
 3   grand_total                     135303 non-null  float64
 4   payment_mode                    135303 non-null  int64  
 5   promo_code                      4305 non-null    object 
 6   vendor_discount_amount          135303 non-null  float64
 7   promo_code_discount_percentage  65880 non-null   float64
 8   is_favorite                     100108 non-null  object 
 9   is_rated                        135303 non-null  object 
 10  vendor_rating                   45220 non-null   float64
 11  driver_rating                   135303 non-null  int64  


In [ ]:
# Count missing values in 'dob' column
train_missing_dob = train_customers['dob'].isnull().sum()
test_missing_dob = test_customers['dob'].isnull().sum()

print(f"Missing values in 'dob' column - Train set: {train_missing_dob}")
print(f"Missing values in 'dob' column - Test set: {test_missing_dob}")


Missing values in 'dob' column - Train set: 31628
Missing values in 'dob' column - Test set: 8920


In [ ]:
# For 'dob', we can fill missing values with the median year. This is a simple imputation strategy.
# First, calculate the median for both train and test sets and print them.
train_median_dob = train_customers['dob'].median()
test_median_dob = test_customers['dob'].median()
print(f"Train median 'dob': {train_median_dob}")
print(f"Test median 'dob': {test_median_dob}\n")

Train median 'dob': 1993.0
Test median 'dob': 1994.0



In [ ]:
#Now fill the missing values
train_customers['dob'].fillna(train_median_dob)
test_customers['dob'].fillna(test_median_dob)


,dob
0,1994.0
1,1994.0
2,1994.0
3,1994.0
4,1997.0
...,...
9763,1994.0
9764,1994.0
9765,1994.0
9766,1994.0


In [ ]:
print(train_customers['dob'].isnull().sum())  # Should print 0
print(test_customers['dob'].isnull().sum())   # Should print 0


0
0


In [ ]:
train_missing_location_type = train_locations['location_type'].isnull().sum()
test_missing_location_type = test_locations['location_type'].isnull().sum()

print(f"Missing values in 'location_type' - Train set: {train_missing_location_type}")
print(f"Missing values in 'location_type' - Test set: {test_missing_location_type}")


Missing values in 'location_type' - Train set: 27209
Missing values in 'location_type' - Test set: 7650


In [ ]:
train_locations['location_type'] = train_locations['location_type'].fillna('Other')
test_locations['location_type'] = test_locations['location_type'].fillna('Other')

In [ ]:
train_missing_lat_lon = train_locations[['latitude', 'longitude']].isnull().any(axis=1).sum()
test_missing_lat_lon = test_locations[['latitude', 'longitude']].isnull().any(axis=1).sum()

print(f"Rows with missing latitude or longitude - Train set: {train_missing_lat_lon}")
print(f"Rows with missing latitude or longitude - Test set: {test_missing_lat_lon}")


Rows with missing latitude or longitude - Train set: 6
Rows with missing latitude or longitude - Test set: 3


In [ ]:
# Drop rows with missing latitude/longitude
train_locations.dropna(subset=['latitude', 'longitude'], inplace=True)
test_locations.dropna(subset=['latitude', 'longitude'], inplace=True)


In [ ]:
# Check missing again after drop
train_missing_after = train_locations[['latitude', 'longitude']].isnull().any(axis=1).sum()
test_missing_after = test_locations[['latitude', 'longitude']].isnull().any(axis=1).sum()

print(f"Missing latitude or longitude after drop - Train set: {train_missing_after}")
print(f"Missing latitude or longitude after drop - Test set: {test_missing_after}")


Missing latitude or longitude after drop - Train set: 0
Missing latitude or longitude after drop - Test set: 0


In [ ]:
datetime_cols = {
    'orders': ['created_at', 'delivery_date', 'order_accepted_time', 'driver_accepted_time',
               'ready_for_pickup_time', 'picked_up_time', 'delivered_time'],
    'train_customers': ['created_at', 'updated_at'],
    'test_customers': ['created_at', 'updated_at'],
    'vendors': ['created_at', 'updated_at']
}

for df_name, cols in datetime_cols.items():
    df = globals()[df_name]
    for col in cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')


In [ ]:
# --- 3. Verify the changes ---
print("\n--- Info for train_customers after cleaning ---")
train_customers.info()

print("\n--- Info for orders after cleaning ---")
orders.info()


--- Info for train_customers after cleaning ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34674 entries, 0 to 34673
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   customer_id  34674 non-null  object        
 1   gender       22520 non-null  object        
 2   dob          34674 non-null  float64       
 3   status       34674 non-null  int64         
 4   verified     34674 non-null  int64         
 5   language     21099 non-null  object        
 6   created_at   34555 non-null  datetime64[ns]
 7   updated_at   34546 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 2.1+ MB

--- Info for orders after cleaning ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135303 entries, 0 to 135302
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          -------------- 

In [ ]:
print(orders['customer_id'].isna().sum())
print(orders['vendor_id'].isna().sum())
print(train_locations['location_number'].isna().sum())
# and so on...


0
0
0


In [ ]:
orders['customer_id'] = orders['customer_id'].astype(str)
train_customers['customer_id'] = train_customers['customer_id'].astype(str)
# Same for vendor_id and LOCATION_NUMBER columns


In [ ]:
print("Before conversion:")
print(orders['customer_id'].dtype)
print(train_customers['customer_id'].dtype)

# Convert
orders['customer_id'] = orders['customer_id'].astype(str)
train_customers['customer_id'] = train_customers['customer_id'].astype(str)

print("\nAfter conversion:")
print(orders['customer_id'].dtype)
print(train_customers['customer_id'].dtype)


Before conversion:
object
object

After conversion:
object
object


In [ ]:
# --- Merge 1: Combine orders with customer information ---
# We use a 'left' merge to ensure we keep all orders, even if a customer's details are missing.
train_df = pd.merge(orders, train_customers, on='customer_id', how='left')




In [ ]:
# --- Merge 2: Add customer location data ---
# We need to match not just the customer, but the specific location number for that order.
# Note: We need to rename the 'location_number' column in train_locations before merging to avoid conflicts.
train_df = pd.merge(train_df, train_locations.rename(columns={'location_number': 'LOCATION_NUMBER'}),
                    on=['customer_id', 'LOCATION_NUMBER'], how='left')




In [ ]:
# --- Merge 3: Add vendor information ---
# The 'id' column in the vendors table corresponds to 'vendor_id' in the orders table.
train_df = pd.merge(train_df, vendors.rename(columns={'id': 'vendor_id'}),
                    on='vendor_id', how='left')




In [ ]:
# --- Verify the final merged DataFrame ---
print("✅ All training data has been merged successfully!")
print("\n--- Info for the final merged training DataFrame ---")
train_df.info()


# Note: You might see suffixes like '_x' and '_y' on column names.
# This happens when you merge dataframes that have columns with the same name (e.g., 'created_at').
# Pandas adds these suffixes to keep them distinct.

✅ All training data has been merged successfully!

--- Info for the final merged training DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135502 entries, 0 to 135501
Data columns (total 94 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   order_id                        135432 non-null  float64       
 1   customer_id                     135502 non-null  object        
 2   item_count                      128577 non-null  float64       
 3   grand_total                     135502 non-null  float64       
 4   payment_mode                    135502 non-null  int64         
 5   promo_code                      4325 non-null    object        
 6   vendor_discount_amount          135502 non-null  float64       
 7   promo_code_discount_percentage  66040 non-null   float64       
 8   is_favorite                     100307 non-null  object        
 9   is_rated             

In [ ]:
display_cols = ['customer_id', 'order_id', 'grand_total', 'vendor_id', 'vendor_rating_x', 'dob', 'latitude_x', 'longitude_x', 'vendor_tag_name']
print(train_df[display_cols].head())


  customer_id  order_id  grand_total  vendor_id  vendor_rating_x     dob  \
0     KL09J9N  163923.0         10.1         84              NaN  1993.0   
1     H5LGGFX  163924.0          8.4         78              NaN  1993.0   
2     CYLZB6T  163925.0         15.0          4              NaN  1993.0   
3     4YKUKYN  163929.0         27.2        157              NaN  1993.0   
4     WDNU30K  163930.0          6.5        160              NaN  1993.0   

   latitude_x  longitude_x                                    vendor_tag_name  
0   -0.090650   -78.580196                   Burgers,Fries,Kids meal,Shawarma  
1    1.733950   -78.795830  Pizzas,Italian,Breakfast,Soups,Pasta,Salads,De...  
2   -0.368541   -78.547354  Arabic,Breakfast,Burgers,Desserts,Free Deliver...  
3    0.421104   -78.640676      Biryani,Desserts,Indian,Rice,Thali,Vegetarian  
4   -0.704372   -78.507665              American,Burgers,Kids meal,Sandwiches  


In [ ]:
# --- 1. Get a count of missing values for each column ---
missing_counts = train_df.isnull().sum()
print("--- Missing Values Count in Merged DataFrame ---")
# Display columns that have at least one missing value
print(missing_counts[missing_counts > 0])


--- Missing Values Count in Merged DataFrame ---
order_id                              70
item_count                          6925
promo_code                        131177
promo_code_discount_percentage     69462
is_favorite                        35195
vendor_rating_x                    90233
preparationtime                    55560
delivery_time                     130370
order_accepted_time                48348
driver_accepted_time               88925
ready_for_pickup_time              51065
picked_up_time                     51446
delivered_time                     49565
delivery_date                      99958
LOCATION_TYPE                      48976
gender                             30754
dob                                 3276
status_x                            3276
verified_x                          3276
language_x                         25039
created_at_y                        3338
updated_at_x                        3363
location_type                          5
latitude

In [ ]:
# --- 1. Drop Columns with Too Many Missing Values ---
# If a column is missing >50% of its data, it's often better to remove it.
cols_to_drop = [
    'promo_code', 'promo_code_discount_percentage', 'delivery_time',
    'driver_accepted_time', 'sunday_from_time2', 'sunday_to_time2',
    'monday_from_time2', 'monday_to_time2', 'tuesday_from_time2', 'tuesday_to_time2',
    'wednesday_from_time2', 'wednesday_to_time2', 'thursday_from_time2', 'thursday_to_time2',
    'friday_from_time2', 'friday_to_time2', 'saturday_from_time2', 'saturday_to_time2'
]
train_df.drop(columns=cols_to_drop, inplace=True)
print(f"✅ Dropped {len(cols_to_drop)} columns with high percentage of missing values.")

✅ Dropped 18 columns with high percentage of missing values.


In [ ]:
# --- 2. Impute Numeric Columns with Median ---
# Median is robust to outliers and is a good choice for vendor ratings, time, etc.
numeric_cols_to_fill = [
    'order_id', 'item_count', 'vendor_rating_x', 'preparationtime',
    'dob', 'status_x', 'verified_x', 'commission'
]
for col in numeric_cols_to_fill:
    if col in train_df.columns:
        median_val = train_df[col].median()
        train_df[col] = train_df[col].fillna(median_val)
print("✅ Imputed key numeric columns with their median.")


✅ Imputed key numeric columns with their median.


In [ ]:
# --- 3. Impute Categorical Columns with Mode ---
categorical_cols_to_fill = [
    'is_favorite', 'LOCATION_TYPE', 'gender', 'language_x',
    'vendor_tag', 'vendor_tag_name', 'primary_tags'
]
for col in categorical_cols_to_fill:
     if col in train_df.columns:
        mode_val = train_df[col].mode()[0]
        train_df[col] = train_df[col].fillna(mode_val)

print("✅ Imputed key categorical columns with their mode.")

✅ Imputed key categorical columns with their mode.


In [ ]:
# --- 4. Drop Rows with Critical Missing Info ---
train_df.dropna(subset=['latitude_x', 'longitude_x', 'order_accepted_time'], inplace=True)
print("✅ Dropped rows where essential data was missing.")

✅ Dropped rows where essential data was missing.


In [ ]:
# --- 5. Final Verification ---
print("\n--- Final Check for Missing Values ---")
missing_counts_after_cleaning = train_df.isnull().sum()
final_missing = missing_counts_after_cleaning[missing_counts_after_cleaning > 0]

if final_missing.empty:
    print("\nNo missing values remain. Your data is clean! 🎉")
else:
    print("\nRemaining missing values:")
    print(final_missing)


--- Final Check for Missing Values ---

Remaining missing values:
ready_for_pickup_time     2746
picked_up_time            3134
delivered_time            1322
delivery_date            87154
created_at_y              2201
updated_at_x              2221
OpeningTime               8919
OpeningTime2              8919
language_y               15185
sunday_from_time1           56
sunday_to_time1             56
tuesday_from_time1          56
tuesday_to_time1            56
thursday_from_time1         56
thursday_to_time1           56
friday_from_time1         1712
friday_to_time1           1712
saturday_from_time1        378
saturday_to_time1          378
dtype: int64


In [ ]:
# --- 1. Drop the delivery_date column ---
# It has too many missing values to be reliable.
if 'delivery_date' in train_df.columns:
    train_df.drop(columns=['delivery_date'], inplace=True)



In [ ]:
# This method fills a NaN with the last valid observation. It's good for sequential data.
time_cols_to_ffill = [
    'ready_for_pickup_time', 'picked_up_time', 'delivered_time',
    'created_at_y', 'updated_at_x'
]
for col in time_cols_to_ffill:
    if col in train_df.columns:
        train_df[col].ffill()
        # In case the very first rows are NaN, bfill will fill them from the next valid one.
        train_df[col].bfill(inplace=True)

In [ ]:
mode_fill_cols = [
    'OpeningTime', 'OpeningTime2', 'language_y', 'sunday_from_time1',
    'sunday_to_time1', 'tuesday_from_time1', 'tuesday_to_time1',
    'thursday_from_time1', 'thursday_to_time1', 'friday_from_time1',
    'friday_to_time1', 'saturday_from_time1', 'saturday_to_time1'
]
for col in mode_fill_cols:
    if col in train_df.columns:
        mode_val = train_df[col].mode()[0]
        train_df[col] = train_df[col].fillna(mode_val)

In [ ]:
print("\n--- Final, Final Check for Missing Values ---")
final_missing_check = train_df.isnull().sum().sum()

if final_missing_check == 0:
    print("\nSuccess! All missing values have been handled. Your data is now completely clean.")
else:
    print(f"\nStill found {final_missing_check} missing values. Please review the remaining columns.")



--- Final, Final Check for Missing Values ---

Success! All missing values have been handled. Your data is now completely clean.


In [ ]:
import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

# STEP 4: FEATURE ENGINEERING
# This part was missed. It creates the columns that caused the error.

print("Running Feature Engineering...")
# 1. Customer Age
current_year = datetime.now().year
train_df['customer_age'] = current_year - train_df['dob']

# 2. Haversine Distance
def haversine_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat2 - lat1)**2 + (lon2 - lon1)**2)

train_df['distance_km'] = haversine_distance(
    train_df['latitude_x'], train_df['longitude_x'],
    train_df['latitude_y'], train_df['longitude_y']
)



Running Feature Engineering...


In [ ]:
# 3. Time-Based Features
train_df['order_hour'] = train_df['created_at_x'].dt.hour
train_df['order_day_of_week'] = train_df['created_at_x'].dt.dayofweek # Monday=0, Sunday=6

# 4. Fill any NaNs that might have been created
train_df['customer_age'].fillna(train_df['customer_age'].median())
train_df['distance_km'].fillna(train_df['distance_km'].median())



,distance_km
38819,209.815604
38820,0.396242
38821,0.534822
38822,0.535882
38823,0.363061
...,...
135148,0.031578
135149,1.117884
135150,0.247597
135151,0.206146


In [ ]:
# 1. Feature Selection
features = [
    'customer_id', 'vendor_id', 'item_count', 'grand_total',
    'vendor_discount_amount', 'is_favorite', 'vendor_rating_x',
    'driver_rating', 'deliverydistance', 'preparationtime',
    'gender', 'customer_age', 'location_type', 'distance_km',
    'order_hour', 'order_day_of_week', 'vendor_category_en',
    'serving_distance', 'is_open', 'prepration_time', 'commission',
    'rank', 'vendor_tag_name', 'city_id'
]

model_df = train_df[features].copy()
model_df['target'] = 1 # All existing orders are positive examples



In [ ]:
# 2. Encoding Categorical Features
categorical_cols = model_df.select_dtypes(include=['object', 'category']).columns
print(f"Columns to be encoded: {list(categorical_cols)}")

for col in categorical_cols:
    le = LabelEncoder()
    model_df[col] = le.fit_transform(model_df[col])


Columns to be encoded: ['customer_id', 'is_favorite', 'gender', 'location_type', 'vendor_category_en', 'vendor_tag_name']


In [ ]:

print(model_df.head())

       customer_id  vendor_id  item_count  grand_total  \
38819          347        231         3.0         10.6   
38820         5461         20         3.0         12.5   
38821        14090          4         2.0         10.6   
38822         1170          4         1.0          9.8   
38823         2473        274         1.0          7.9   

       vendor_discount_amount  is_favorite  vendor_rating_x  driver_rating  \
38819                     0.0            0              0.0              0   
38820                     0.0            0              0.0              0   
38821                     0.0            0              0.0              0   
38822                     0.0            1              5.0              5   
38823                     0.0            0              0.0              0   

       deliverydistance  preparationtime  gender  customer_age  location_type  \
38819               0.0             45.0       6          32.0              2   
38820               

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [ ]:
# --- 1. Create Negative Samples ---
print("Creating negative samples...")
all_customers = model_df['customer_id'].unique()
all_vendors = model_df['vendor_id'].unique()
existing_pairs = set(zip(model_df['customer_id'], model_df['vendor_id']))

negative_samples = []
# Create a 1:1 ratio of positive to negative samples
num_negative_samples = len(model_df)

while len(negative_samples) < num_negative_samples:
    customer = np.random.choice(all_customers)
    vendor = np.random.choice(all_vendors)
    if (customer, vendor) not in existing_pairs:
        # For negative samples, we use typical or neutral values.
        sample = {
            'customer_id': customer,
            'vendor_id': vendor,
            'item_count': model_df['item_count'].median(),
            'grand_total': 0,
            'vendor_discount_amount': 0,
            'is_favorite': 0,
            'vendor_rating_x': model_df['vendor_rating_x'].median(),
            'driver_rating': 0,
            'deliverydistance': model_df['deliverydistance'].median(),
            'preparationtime': model_df['preparationtime'].median(),
            'gender': model_df['gender'].mode()[0],
            'customer_age': model_df['customer_age'].median(),
            'location_type': model_df['location_type'].mode()[0],
            'distance_km': model_df['distance_km'].median(),
            'order_hour': 12, # A neutral hour of the day
            'order_day_of_week': 3, # A neutral day of the week
            'vendor_category_en': model_df['vendor_category_en'].mode()[0],
            'serving_distance': model_df['serving_distance'].median(),
            'is_open': 1,
            'prepration_time': model_df['prepration_time'].median(),
            'commission': model_df['commission'].median(),
            'rank': model_df['rank'].median(),
            'vendor_tag_name': model_df['vendor_tag_name'].mode()[0],
            'city_id': model_df['city_id'].mode()[0],
            'target': 0 # Target is 0 for non-orders
        }
        negative_samples.append(sample)
        # Add to set to avoid creating duplicate negative samples
        existing_pairs.add((customer, vendor))

negative_df = pd.DataFrame(negative_samples)
print(f"✅ Created {len(negative_df)} negative samples.")


Creating negative samples...
✅ Created 87154 negative samples.


In [ ]:
# --- 2. Combine and Prepare Final Dataset ---
print("Combining positive and negative samples...")
final_df = pd.concat([model_df, negative_df], ignore_index=True)
# Shuffle the dataset to mix positive and negative samples
final_df = final_df.sample(frac=1).reset_index(drop=True)

# Define features (X) and target (y)
X = final_df.drop(columns=['target'])
y = final_df['target']

# Split data for training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print("✅ Final dataset prepared and split.")

Combining positive and negative samples...
✅ Final dataset prepared and split.


In [ ]:
# --- 3. Train the LightGBM Model ---
print("Training LightGBM model...")

# Define model parameters
params = {
    'objective': 'binary',
    'metric': 'auc', # AUC is a great metric for this type of problem
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'seed': 42,
    'n_jobs': -1,
    'verbose': -1,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
}

model = lgb.LGBMClassifier(**params)

# Train the model with a callback for early stopping
model.fit(X_train, y_train,
          eval_set=[(X_val, y_val)],
          eval_metric='auc',
          callbacks=[lgb.early_stopping(100, verbose=True)])

print("\n🎉 Model training complete! You now have a trained recommendation engine.")

Training LightGBM model...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	valid_0's auc: 1

🎉 Model training complete! You now have a trained recommendation engine.


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

# --- 1. Load All Necessary Data ---
print("Loading all necessary data...")
test_customers = pd.read_csv('/content/drive/MyDrive/recomendation system/Assignment/Test/test_customers.csv')
test_locations = pd.read_csv('/content/drive/MyDrive/recomendation system/Assignment/Test/test_locations.csv')
vendors = pd.read_csv('/content/drive/MyDrive/recomendation system/Assignment/Train/vendors.csv')

Loading all necessary data...


In [ ]:
# --- 2. Create All Customer-Vendor Combinations for the Test Set ---
print("Generating test combinations...")
all_vendor_ids = vendors['id'].unique()
test_locations['key'] = 1
all_vendors_df = pd.DataFrame({'vendor_id': all_vendor_ids, 'key': 1})

Generating test combinations...


In [ ]:
# Master test dataframe
test_df = pd.merge(test_locations, all_vendors_df, on='key').drop('key', axis=1)
test_df = pd.merge(test_df, test_customers, on='customer_id', how='left')
test_df = pd.merge(test_df, vendors.rename(columns={'id': 'vendor_id'}), on='vendor_id', how='left')
print(f"Created {len(test_df)} combinations for prediction.")


Created 1673600 combinations for prediction.


In [ ]:
# --- 3. Create Placeholder Columns & Apply Feature Engineering (FIXES THE ERROR) ---
print("Applying feature engineering and creating placeholder columns...")

# Create columns from the training set that don't exist in the test set
# and fill them with neutral/median values. This is CRITICAL.
for col in features:
    if col not in test_df.columns and col != 'target':
        # Use a sensible default, like the median from the training data (model_df)
        test_df[col] = model_df[col].median()

# Now, apply the same feature engineering as before
current_year = datetime.now().year
test_df['customer_age'] = current_year - test_df['dob']
test_df['distance_km'] = np.sqrt((test_df['latitude_y'] - test_df['latitude_x'])**2 + (test_df['longitude_y'] - test_df['longitude_x'])**2)
test_df['order_hour'] = 18  # Assume a peak evening hour
test_df['order_day_of_week'] = 5 # Assume a Friday

print("✅ Feature engineering for test set is complete.")

Applying feature engineering and creating placeholder columns...
✅ Feature engineering for test set is complete.


In [ ]:
# --- 4. Clean, Fill NaNs, and Encode ---
print("Cleaning and encoding test data...")
final_test_df = test_df[features].copy() # This will now work without a KeyError

# Fill any remaining NaNs
for col in final_test_df.columns:
    if final_test_df[col].isnull().any():
        if final_test_df[col].dtype.kind in 'ifc': # Numeric columns
            fill_value = model_df[col].median()
        else: # Categorical columns
            fill_value = model_df[col].mode()[0]
        final_test_df[col].fillna(fill_value, inplace=True)

# Encode categorical columns using the label encoders fitted on the training data
for col in categorical_cols:
    # Load the encoder fitted on the original training data
    le = LabelEncoder()
    le.fit(model_df[col]) # Fit on the original model_df's column

    # Transform the test data, handling any new, unseen labels
    # We map unseen labels to a special value like -1 or the most frequent class
    final_test_df[col] = final_test_df[col].map(lambda s: s if s in le.classes_ else le.classes_[0])
    final_test_df[col] = le.transform(final_test_df[col])

print("✅ Cleaning and encoding for test set is complete.")


Cleaning and encoding test data...


/tmp/ipython-input-945358993.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_test_df[col].fillna(fill_value, inplace=True)
/tmp/ipython-input-945358993.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

✅ Cleaning and encoding for test set is complete.


In [ ]:
# --- 5. Make Predictions ---
print("Making predictions...")
# Ensure column order matches the training data (X)
final_test_df = final_test_df[X.columns]
predictions = model.predict_proba(final_test_df)[:, 1] # Probability of '1'

Making predictions...


In [ ]:
# --- 6. Generate Submission File ---
print("Generating submission file...")
submission_df = test_df[['customer_id', 'location_number', 'vendor_id']].copy()
submission_df['CID X LOC_NUM X VENDOR'] = submission_df['customer_id'] + ' X ' + submission_df['location_number'].astype(str) + ' X ' + submission_df['vendor_id'].astype(str)
submission_df['target'] = (predictions > 0.5).astype(int) # Convert probability to binary 0 or 1


Generating submission file...


In [ ]:
# Save to CSV
submission_df[['CID X LOC_NUM X VENDOR', 'target']].to_csv('submission.csv', index=False)


print(submission_df[['CID X LOC_NUM X VENDOR', 'target']].head())

  CID X LOC_NUM X VENDOR  target
0        Z59FTQD X 0 X 4       1
1       Z59FTQD X 0 X 13       1
2       Z59FTQD X 0 X 20       1
3       Z59FTQD X 0 X 23       1
4       Z59FTQD X 0 X 28       1
